In [15]:
import os, sys, time
import xarray as xr
import pandas as pd
import datetime as dt


In [16]:
datadir = "/p/scratch/deepacf/video_prediction_shared_folder/preprocessedData/T2monthly"

datafile = "1970-1999_t2m.nc"

datafile= os.path.join(datadir, datafile)

datafile="/p/scratch/deepacf/video_prediction_shared_folder/preprocessedData/T2monthly/t2m_1970_1999.nc"

In [17]:
with xr.open_dataset(datafile) as dfile:
    t2m_all = dfile["var167"]
    coords = t2m_all.coords

In [18]:
ntimes = len(coords["time"])

t2m_all = t2m_all.chunk({"time": ntimes, "lat":100, "lon":100})

In [19]:
# define a function with the hourly calculation:
def hour_mean(x):
     return x.groupby('time.hour').mean('time')

time0 = time.time()
t2m_hourly = t2m_all.groupby("time.month").apply(hour_mean)

print("Registering averaging took {0:.2f}".format(time.time()-time0))

#print(t2m_hourly.values)

print("Performing averaging took {0:.2f}".format(time.time()-time0))



/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    

Registering averaging took 1.08
Performing averaging took 1.08


/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/p/software/hdfml/stages/2020/software/Jupyter/2020.2.6-gcccoremkl-9.3.0-2020.2.254-Python-3.8.5/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    

In [20]:
t2m_hourly = t2m_hourly.compute()

This works, but it takes about 3 minutes to process 30 years of data. <br>
However, the same operation is possible with CDO and only takes 36s to finish on Juwels. <br> 
The two following shell commands (after loading CDO 1.9.8 and ecCodes 2.18.0) are:
```
clim_files=($(for year in {1991..2020}; do echo "${year}_t2m.grb"; done))
cdo -t ecmwf -f nc ensavg ${clim_files[@]} mutilyears_1991-2020.nc
```
In the following, we check the correctness of the data by computing the difference btween the data from a CDO-generated file against the data produced above. We choose the mean temperature in January at 12 UTC as an example.

In [40]:
datafile_cdo = os.path.join(datadir, "mutilyears_1970-1999.nc")

with xr.open_dataset(datafile_cdo) as dfile:
    t2m_hourly_cdo = dfile["T2M"]
   

In [41]:
import numpy as np
test1 = t2m_hourly.sel(month=1, hour=12)
test2 = t2m_hourly_cdo.sel(time="1979-01-01 12:00")

diff = np.abs(test1-test2)

print(np.max(diff))

<xarray.DataArray ()>
array(0.00097656, dtype=float32)
Coordinates:
    hour     int64 12
    month    int64 1
    time     datetime64[ns] 1979-01-01T12:00:00


Thus, the maximum difference is in the $\mathcal{O} (10^{-3})$ which can be neglected for our application.